In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

**Read dataset**

In [9]:
filtered_clothing = pd.read_csv('filtered_clothing_dataset.csv')

**Split dataset in training and testing set**

In [10]:
eighty_pct = int(0.8*len(filtered_clothing))
  
train_set = filtered_clothing.iloc[:(eighty_pct-1), :] 
test_set = filtered_clothing.iloc[eighty_pct:, :] 
  
train_set.to_csv("train_data.csv")
test_set.to_csv("test_data.csv")

**Loading the dataset**

In [11]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

In [12]:
# image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data['id'] = train_data['id'].astype(str) + '.jpg'
test_data['id'] = test_data['id'].astype(str) + '.jpg'

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='filtered_images',
    x_col="id",
    y_col="subCategory",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='filtered_images',
    x_col="id",
    y_col="subCategory",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 22615 validated image filenames belonging to 7 classes.
Found 5655 validated image filenames belonging to 7 classes.


**Defining the CNN model**

In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(7, activation='softmax')  # 7 categories in the dataset
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

C:\Users\catal\anaconda3\envs\fashion_product_images_small\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       9,437,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,682,119 (36.93 MB)

 Trainable params: 9,682,119 (36.93 MB)

 Non-trainable params: 0 (0.00 B)

**Model training**

In [15]:
# number of training and validation steps
train_steps = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size

# model training
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=10,  # adjust as needed
    validation_data=validation_generator,
    validation_steps=validation_steps)

Epoch 1/10


C:\Users\catal\anaconda3\envs\fashion_product_images_small\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


706/706 ━━━━━━━━━━━━━━━━━━━━ 1146s 2s/step - accuracy: 0.8112 - loss: 0.5776 - val_accuracy: 0.9364 - val_loss: 0.1930
Epoch 2/10
  1/706 ━━━━━━━━━━━━━━━━━━━━ 12:26 1s/step - accuracy: 0.9375 - loss: 0.1215

C:\Users\catal\anaconda3\envs\fashion_product_images_small\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


706/706 ━━━━━━━━━━━━━━━━━━━━ 1s 403us/step - accuracy: 0.9375 - loss: 0.0608 - val_accuracy: 0.9565 - val_loss: 0.0591
Epoch 3/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 885s 1s/step - accuracy: 0.9382 - loss: 0.1917 - val_accuracy: 0.9547 - val_loss: 0.1604
Epoch 4/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 1s 298us/step - accuracy: 0.9688 - loss: 0.0257 - val_accuracy: 1.0000 - val_loss: 0.0172
Epoch 5/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 888s 1s/step - accuracy: 0.9510 - loss: 0.1439 - val_accuracy: 0.9627 - val_loss: 0.1326
Epoch 6/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 1s 299us/step - accuracy: 1.0000 - loss: 0.0090 - val_accuracy: 0.8696 - val_loss: 0.1008
Epoch 7/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 888s 1s/step - accuracy: 0.9618 - loss: 0.1138 - val_accuracy: 0.9513 - val_loss: 0.1622
Epoch 8/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 1s 299us/step - accuracy: 0.9688 - loss: 0.0514 - val_accuracy: 0.9130 - val_loss: 0.1071
Epoch 9/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 941s 1s/step - accuracy: 0.9670 - loss: 0.0965 - val_accuracy: 0

**Model evaluation**

In [16]:
test_loss, test_accuracy = model.evaluate(validation_generator, steps=validation_steps)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

176/176 ━━━━━━━━━━━━━━━━━━━━ 63s 357ms/step - accuracy: 0.9534 - loss: 0.1462
Test Loss: 0.14031553268432617
Test Accuracy: 0.955078125


**Save model**

In [17]:
model.save('CNN_model_11032024.keras')
print("Saving the model as CNN_model_11032024")

Saving the model as CNN_model_11032024
